# apache clipper 
## demo 1 : build helloworld and test

## 1. build up Clipper Management

In [4]:
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as python_deployer

引用函示庫

In [5]:
clipper_conn = ClipperConnection(DockerContainerManager())


初始 clipper_conn 與 docker container 連線 ( docker 服務需啟動 ) 

In [6]:
clipper_conn.start_clipper()

19-09-05:08:24:18 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
19-09-05:08:24:28 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpsfst_x57.yml
19-09-05:08:24:32 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


啟動clipper manager (listen on 1337 ) 

## 2. Register 微服務@docker

In [7]:
clipper_conn.register_application(
    name="hello-world", 
    input_type="doubles", 
    default_output="-1.0", 
    slo_micros=100000)


19-09-05:08:24:32 INFO     [clipper_admin.py:236] [default-cluster] Application hello-world was successfully registered


註冊一個名為hello-world 的 application 微服務

In [8]:
clipper_conn.get_all_apps()


['hello-world']

測試看看是否註冊成功

In [9]:
def feature_sum(xs):
    return [str(sum(x)) for x in xs]


撰寫一個簡單的累加應用function

In [10]:
python_deployer.deploy_python_closure(clipper_conn, 
                                      name="waue-sum-model", 
                                      version=2, 
                                      input_type="doubles", 
                                      func=feature_sum,
                                      registry="waue0920"
                                     )


19-09-05:08:24:32 INFO     [deployer_utils.py:41] Saving function to /tmp/tmp27pnohdiclipper
19-09-05:08:24:32 INFO     [deployer_utils.py:51] Serialized and supplied predict function
19-09-05:08:24:32 INFO     [python.py:192] Python closure saved
19-09-05:08:24:32 INFO     [python.py:206] Using Python 3.6 base image
19-09-05:08:24:32 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmp27pnohdiclipper
19-09-05:08:24:34 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
19-09-05:08:24:34 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
19-09-05:08:24:34 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmp27pnohdiclipper /model/
19-09-05:08:24:34 INFO     [clipper_admin.py:539] [default-cluster]  ---> dab656fc0f3f
19-09-05:08:24:34 INFO     [clipper_admin.py:539] [default-cluster] Successfully built dab656fc0f3f
19-09-05:08:24:34 INFO   

這個階段會執行比較久的時間，將feature_sum 服務駐進model container image中，參數 name, version, registry 是docker hub 上的相關參數，用來上傳這個container到docker hub，因此 docker hub 相關的設定要對，也要先確定此台server 能 docker login 到docker hub上，否則這段會出錯。

<table border="5" align="left">
    <tr align="center">
        <td>參數名稱與付值</td><td>說明</td>
    </tr>
    <tr align="left">
        <td>clipper_conn</td><td>原先建立的連線</td>
    </tr>
    <tr align="left"><td>name="waue-sum-model"</td><td>上傳到 docker hub上的名稱</td></tr>
    <tr align="left"><td>version=2</td><td>上傳到 docker hub上的版本</td></tr>
    <tr align="left"><td>registry="waue0920"</td><td>docker hub 登入的帳號</td></tr>
    <tr align="left"><td>func=feature_sum</td><td>微服務 function 名稱</td></tr>
    <tr align="left"><td>input_type="doubles"</td><td>prediction階段時的輸入型態</td></tr>
</table>

In [11]:
clipper_conn.link_model_to_app(app_name="hello-world", model_name="waue-sum-model")

19-09-05:08:24:52 INFO     [clipper_admin.py:303] [default-cluster] Model waue-sum-model is now linked to application hello-world


將原本微服務 hello-world 與 model container image : waue-sum-model 鏈結在一起

## 3. prediction via restful api

In [13]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://localhost:1337/hello-world/predict", 
              headers=headers, data=json.dumps({"input": [ float(x) for x in range(1,99)]})).json()

{'query_id': 1, 'output': 4851.0, 'default': False}

用restful api 丟一個1~9999的list給微服務，可以看到直接回傳結果

用 curl :
<code> curl -X POST "http://203.145.220.208:1337/hello-world/predict" -H "accept: application/json" -d "{\"input\":[1.0,2.0]}" </code>

## demo1 : reconnect to clipper manager

In [1]:
from clipper_admin import ClipperConnection, DockerContainerManager
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.connect()

19-09-05:08:14:33 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at localhost:1337


In [2]:
clipper_conn.get_all_apps()


['face-gender-orangejuice571',
 'face-gender-orangejuice813',
 'face-gender-orangejuice403',
 'face-gender-orange',
 'face-gender-orangejuice506',
 'face-gender-orangejuice368',
 'face-app',
 'face-gender',
 'hello-world',
 'face-gender-orangejuice773']

## demo1 : close services 

In [3]:
clipper_conn.stop_all()

19-09-05:08:17:02 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers
